In [152]:
import requests
from pprint import pprint
import pandas as pd
import datetime as dt
import numpy as np
from bs4 import BeautifulSoup as bs

In [8]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 


In [44]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

# Web page for the income by zipcode data

In [45]:
AustinIncome_url = 'http://zipatlas.com/us/tx/austin/zip-code-comparison/median-household-income.htm'

# This web page is pretty slow. It also updates upon entry to the web page. 
# So we have to wait for the data to show up to get consistent behavior.

In [46]:
browser.visit(AustinIncome_url)
browser.is_element_present_by_id('ct100_ContentPlaceHolder1_final_content', wait_time=10)
html = browser.html
soup = bs(html, 'html.parser')

In [47]:
browser.quit()

In [79]:
incometable = soup.select_one("table:nth-of-type(16)")
income_df = pd.read_html(str(incometable), header=0)[0]
chg_cols = {"Zip Code" : "ZipCode", "Avg. Income/H/hold": "HouseholdIncome", "National Rank" :"NationalRank"}
income_df.rename(inplace=True, columns = chg_cols)
income_df.head()

,#,ZipCode,Location,City,Population,HouseholdIncome,NationalRank
0,1.0,78730,"30.363632, -97.850355","Austin, Texas",4885,"$128,524.00",#58
1,2.0,78732,"30.382536, -97.894904","Austin, Texas",3629,"$103,951.00",#209
2,3.0,78739,"30.178839, -97.889064","Austin, Texas",8643,"$102,707.00",#227
3,4.0,78738,"30.327510, -97.989040","Austin, Texas",2840,"$102,295.00",#231
4,5.0,78733,"30.323857, -97.894860","Austin, Texas",8716,"$102,239.00",#232


# Build the db from Pandas with this table
# Right now, this is just in memory, not stored. Need to decide. We could just 
# recreate every time the page is loaded.

In [140]:
engine = create_engine('sqlite://', echo=False)

In [141]:
income_df.to_sql("austin_income", con=engine, dtype={col_name: String for col_name in income_df})

In [139]:
# engine.execute("drop table austin_income")

In [145]:
zipcodes = engine.execute('select * from austin_income').fetchall()

In [146]:
print(type(zipcodes[0]))
print(zipcodes[0])

<class 'sqlalchemy.engine.result.RowProxy'>
(0, '1.0', '78730', '30.363632, -97.850355', 'Austin, Texas', '4885', '$128,524.00', '#58')


In [147]:
print(zipcodes[0].ZipCode)

78730


# Now to get the animal data

In [136]:

# client = Socrata("data.austintexas.gov",
#                  "ks9jD1z8CH7X1yKYGc82uYO4z",
#                  username="dvohmart@gmail.com",
#                  password="AustinTexas1220")
austin_url = "https://data.austintexas.gov/resource/"
dataset = "9t4d-g238.json"
api_key = "ks9jD1z8CH7X1yKYGc82uYO4z"

query = austin_url + dataset + "?$where=datetime > '2019-01-01T00:00:00.000'&$limit=10000"
print(query)
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = requests.get(query)
results_json = results.json()
# Convert to pandas DataFrame
outcome_df = pd.DataFrame.from_records(results_json)
outcome_df.head()
# print(len(outcome_df))

https://data.austintexas.gov/resource/9t4d-g238.json?$where=datetime > '2019-01-01T00:00:00.000'&$limit=10000


,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,1 year,A777234,Dog,Pit Bull Mix,Blue/White,2017-11-25T00:00:00.000,2019-01-02T00:00:00.000,2019-01-02T00:00:00.000,Tyson,Partner,Transfer,Neutered Male
1,3 years,A783891,Dog,Border Terrier,Black/Tan,2015-11-08T00:00:00.000,2019-01-16T11:28:00.000,2019-01-16T11:28:00.000,Elizabeth T,Foster,Adoption,Spayed Female
2,10 months,A788493,Dog,Standard Poodle/Labrador Retriever,Cream,2018-04-03T00:00:00.000,2019-02-07T13:12:00.000,2019-02-07T13:12:00.000,NaN,Partner,Transfer,Neutered Male
3,4 years,A789594,Dog,Chihuahua Longhair Mix,Brown/Black,2015-02-24T00:00:00.000,2019-02-26T19:33:00.000,2019-02-26T19:33:00.000,Morena,NaN,Adoption,Spayed Female
4,10 years,A566568,Dog,Harrier Mix,Brown/Black,2009-02-12T00:00:00.000,2019-02-23T11:33:00.000,2019-02-23T11:33:00.000,Duck,NaN,Adoption,Neutered Male


In [137]:
dataset = "fdzn-9yqv.json"
query = austin_url + dataset + "?$where=datetime > '2019-01-01T00:00:00.000'&$limit=10000"
print(query)
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = requests.get(query)
results_json = results.json()
# Convert to pandas DataFrame
intake_df = pd.DataFrame.from_records(results_json)
intake_df.head()
# print(len(intake_df))

https://data.austintexas.gov/resource/fdzn-9yqv.json?$where=datetime > '2019-01-01T00:00:00.000'&$limit=10000


,age_upon_intake,animal_id,animal_type,breed,color,datetime,datetime2,found_location,intake_condition,intake_type,name,sex_upon_intake
0,2 years,A786884,Dog,Beagle Mix,Tricolor,2019-01-03T16:19:00.000,2019-01-03T16:19:00.000,2501 Magin Meadow Dr in Austin (TX),Normal,Stray,*Brock,Neutered Male
1,3 days,A787254,Dog,Black Mouth Cur Mix,Black/White,2019-01-10T10:19:00.000,2019-01-10T10:19:00.000,2000 Man O War in Travis (TX),Normal,Stray,*Sprinkles,Intact Female
2,7 years,A760116,Dog,German Shepherd,Brown/Black,2019-01-07T12:07:00.000,2019-01-07T12:07:00.000,Austin (TX),Normal,Owner Surrender,Jasmine,Spayed Female
3,3 years,A789406,Cat,Domestic Shorthair Mix,Brown Tabby,2019-02-21T14:55:00.000,2019-02-21T14:55:00.000,Austin (TX),Normal,Public Assist,Liliana,Intact Female
4,4 years,A787054,Dog,Labrador Retriever Mix,Black/White,2019-01-24T15:17:00.000,2019-01-24T15:17:00.000,Austin (TX),Normal,Owner Surrender,*Rexx,Neutered Male


In [148]:
intake_df.to_sql("austin_animal_intake", con=engine)

In [153]:
intake = engine.execute('select * from austin_animal_intake').fetchall()
pprint(intake)

[(0, '2 years', 'A786884', 'Dog', 'Beagle Mix', 'Tricolor', '2019-01-03T16:19:00.000', '2019-01-03T16:19:00.000', '2501 Magin Meadow Dr in Austin (TX)', 'Normal', 'Stray', '*Brock', 'Neutered Male'),
 (1, '3 days', 'A787254', 'Dog', 'Black Mouth Cur Mix', 'Black/White', '2019-01-10T10:19:00.000', '2019-01-10T10:19:00.000', '2000 Man O War in Travis (TX)', 'Normal', 'Stray', '*Sprinkles', 'Intact Female'),
 (2, '7 years', 'A760116', 'Dog', 'German Shepherd', 'Brown/Black', '2019-01-07T12:07:00.000', '2019-01-07T12:07:00.000', 'Austin (TX)', 'Normal', 'Owner Surrender', 'Jasmine', 'Spayed Female'),
 (3, '3 years', 'A789406', 'Cat', 'Domestic Shorthair Mix', 'Brown Tabby', '2019-02-21T14:55:00.000', '2019-02-21T14:55:00.000', 'Austin (TX)', 'Normal', 'Public Assist', 'Liliana', 'Intact Female'),
 (4, '4 years', 'A787054', 'Dog', 'Labrador Retriever Mix', 'Black/White', '2019-01-24T15:17:00.000', '2019-01-24T15:17:00.000', 'Austin (TX)', 'Normal', 'Owner Surrender', '*Rexx', 'Neutered Male

 (1367, '9 months', 'A788920', 'Cat', 'Domestic Shorthair Mix', 'Black', '2019-02-11T15:25:00.000', '2019-02-11T15:25:00.000', 'Austin (TX)', 'Normal', 'Owner Surrender', 'Mojo', 'Intact Female'),
 (1368, '5 years', 'A786775', 'Cat', 'Domestic Shorthair Mix', 'Brown Tabby', '2019-01-01T13:43:00.000', '2019-01-01T13:43:00.000', '12310 Marogot Run in Austin (TX)', 'Normal', 'Owner Surrender', 'Babydoll', 'Neutered Male'),
 (1369, '2 years', 'A789883', 'Dog', 'English Bulldog Mix', 'Brown Brindle/White', '2019-03-01T12:00:00.000', '2019-03-01T12:00:00.000', 'Shoal Creek Boulevard And West 39Th Street in Austin (TX)', 'Normal', 'Stray', 'Drax', 'Neutered Male'),
 (1370, '1 year', 'A788671', 'Dog', 'German Shepherd Mix', 'Red', '2019-02-06T13:27:00.000', '2019-02-06T13:27:00.000', 'Austin (TX)', 'Normal', 'Owner Surrender', 'Lulu', 'Intact Female'),
 (1371, '0 years', 'A788180', 'Dog', 'Pit Bull Mix', 'White', '2019-01-27T17:45:00.000', '2019-01-27T17:45:00.000', 'Austin (TX)', 'Normal', 'S

 (2367, '1 year', 'A790086', 'Cat', 'Domestic Shorthair Mix', 'Blue Tabby', '2019-03-04T16:03:00.000', '2019-03-04T16:03:00.000', '10908 Jones Road in Manor (TX)', 'Normal', 'Stray', None, 'Unknown'),
 (2368, '1 year', 'A789995', 'Cat', 'Domestic Shorthair Mix', 'Calico', '2019-03-02T17:41:00.000', '2019-03-02T17:41:00.000', '4300 Duval Road in Austin (TX)', 'Normal', 'Stray', 'Tigress', 'Intact Female'),
 (2369, '5 months', 'A790097', 'Dog', 'Chihuahua Shorthair Mix', 'Tricolor', '2019-03-04T17:00:00.000', '2019-03-04T17:00:00.000', '200 West Wonsley Drive in Austin (TX)', 'Normal', 'Stray', None, 'Intact Male'),
 (2370, '1 year', 'A784467', 'Dog', 'Pit Bull Mix', 'Fawn/White', '2019-03-04T16:16:00.000', '2019-03-04T16:16:00.000', 'Austin (TX)', 'Normal', 'Owner Surrender', '*Lilac', 'Spayed Female'),
 (2371, '1 month', 'A789941', 'Cat', 'Domestic Shorthair Mix', 'Tortie', '2019-03-02T12:27:00.000', '2019-03-02T12:27:00.000', '8907 Parkfield Drive in Austin (TX)', 'Normal', 'Owner Sur

 (3366, '4 years', 'A791227', 'Dog', 'Siberian Husky Mix', 'Black/White', '2019-03-23T10:01:00.000', '2019-03-23T10:01:00.000', '3909 North Interstate 35 Northbound in Austin (TX)', 'Normal', 'Public Assist', 'Frank', 'Neutered Male'),
 (3367, '4 years', 'A791061', 'Dog', 'Alaskan Malamute Mix', 'Black/White', '2019-03-20T18:21:00.000', '2019-03-20T18:21:00.000', 'Edge Creek Drive And Hidden Brook Court in Austin (TX)', 'Normal', 'Stray', '*Tundra', 'Intact Male'),
 (3368, '2 years', 'A791304', 'Cat', 'Domestic Shorthair Mix', 'Brown Tabby', '2019-03-24T12:02:00.000', '2019-03-24T12:02:00.000', '10615 Razil Court in Austin (TX)', 'Normal', 'Stray', None, 'Unknown'),
 (3369, '1 month', 'A791254', 'Cat', 'Domestic Shorthair Mix', 'Blue Tabby/White', '2019-03-23T15:06:00.000', '2019-03-23T15:06:00.000', '8301 Annabelle Drive in Travis (TX)', 'Normal', 'Owner Surrender', '*Mouse', 'Intact Male'),
 (3370, '2 years', 'A790853', 'Dog', 'Pit Bull Mix', 'Brown Brindle', '2019-03-18T15:05:00.000

In [ ]:
# engine.execute("drop table austin_income")